## Dependencies

In [1]:
# Confirmed needed dependencies
import pandas as pd
import sys

# Dependencies for mapping
import gmplot

# Dependency for Heat Mapper
import gmaps

from config import google_API_Key

from ipywidgets.embed import embed_minimal_html

!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK


## File inputs/outputs

In [2]:
randLatLon_csv = "./Data/Archived/randomLatLon.csv" 
addressList_csv = "./Data/Archived/addressList.csv"
masterDataCLEAN_csv = "./Data/masterDataCLEAN.csv"
crime_df = pd.read_csv('./Data/crimeFixedLatLon.csv')
crimeSev_df = pd.read_csv('./Data/crimeDescSeverity.csv')

## Helpful Functions

In [ ]:
#############################
##### VALERIE'S BLOCKS #####
###########################

# Funtion for reading CSV in as DataFrame
def csvDF(oldCSVfilepath):
    csvIN = pd.read_csv(oldCSVfilepath)
    DF = pd.DataFrame(csvIN)
    return DF

# Function for converting DataFrame to CSV
def DFcsv(dataframe, newCSVfilepath):
    dataframe.to_csv(newCSVfilepath, index=False, header=True)
    print(f"Successfully written to '{newCSVfilepath}'")
    
# Function for reading in csv, checking for headers, and appending if appropriate
def csvDFappend(oldCSVfilepath, newColumn):
    csvIN = pd.read_csv(oldCSVfilepath)
    DF = pd.DataFrame(csvIN)
    # Checking to ensure new header name does not match any current headers
    colNames = DF.columns
    for value in colNames:
        if value == newColumn:
            print("Cannot append column that matches an existing column name")
            return DF
    # Check to ensure length of newColumn matches length of current dataframe columns
    if len(newColumn) != len(DF):
        print("Cannot append column that is not the same length as existing dataframe")
        return DF
    # Append newColumn to Dataframe
    DF[newColumn] = newColumn
    return DF

## Heat Mapper

In [34]:
# Troy's section


gmaps.configure(api_key=google_API_Key)

In [ ]:
# This cell uses gmaps library to create a google heat map from the data in a master data file.
# The masterData csv file is taken as input
# The lat and lon columns are taken as the coordinates for hte heatmap 
# The user specified column is taken as the weighting valies fo each coordinate point

df = masterData_df
columnToMap = 'valueToMap'
max_intensity = df[columnToMap].max()

fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(df[['lat', 'lon']], weights=df[columnToMap], max_intensity=max_intensity, point_radius=10.0)
fig.add_layer(heatmap_layer)
fig

In [ ]:
#####This is currently not working as a fucntion. for expediency I suggest we past teh heatmapper code into a cell as needed

# # this is a function version of the cell above
# # the function takes columnToMap as the weights for the points defined by 'lat' and 'lon' columns in the dataframe
# # the dataframe can be included as a parameter, if it is not included masterData_df is assumed

# def heatMapper(columnToMap, df = masterData_df):
    
#     max_intensity = df[columnToMap].max()
    
#     fig = gmaps.figure()
#     heatmap_layer = gmaps.heatmap_layer(df[['lat', 'lon']], weights=df[columnToMap], max_intensity=max_intensity, point_radius=10.0)
#     fig.add_layer(heatmap_layer)

#     return;

In [ ]:
heatMapper(columnToMap = 'valueToMap')
fig

## Commute Time Analysis

In [3]:
# Troy's section
# This section will prepare the data and display on a heatmap

# read in masterdata file into dataframe

data_df = pd.read_csv(masterDataCLEAN_csv)

min = data_df.commuteTime.min()
max = data_df.commuteTime.max()

data_df["CommuteScore"] =100-((data_df.commuteTime-min)*(100/(max-min)))

data_df.head()


,Zillow ID,address,zipcode,alat,alon,valuation,sqft,value sqft,neighborhood,tractCode,countyFips,stateFips,commuteTime,CommuteScore
0,29498621,8113 West Gate Boulevard,78745,30.197589,-97.828106,265572.0,940.0,282.52,Cherry Creek,1728,453,48,27.7,46.048110
1,29440641,2109 Red Stone Lane,78727,30.418454,-97.696190,249696.0,1359.0,183.74,Tomanet Estates,1848,453,48,21.4,67.697595
2,29345444,9 Glen Rock Drive,78738,30.341187,-97.984049,472040.0,2896.0,163.00,NaN,1773,453,48,30.8,35.395189
3,29613796,8333 Alvin High Lane,78729,30.460441,-97.761118,307384.0,1983.0,155.01,Hunter's Chase,20411,491,48,27.0,48.453608
4,29358809,5300 Valburn Circle,78731,30.374096,-97.775259,941669.0,4376.0,215.19,North Cat Mountain,1705,453,48,20.3,71.477663


In [4]:
# Creating a heat map for the Commute scores

df = data_df
columnToMap = 'CommuteScore'
point_radius = 5


max_intensity = df[columnToMap].max()

figure_layout = {'width': '500px', 'margin': '0 auto 0 auto'}


fig = gmaps.figure(layout=figure_layout)
heatmap_layer = gmaps.heatmap_layer(df[['alat', 'alon']], weights=df[columnToMap], max_intensity=max_intensity, point_radius=point_radius)
fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px', margin='0 auto 0 auto', width='500px'))

In [5]:
# Change the heat map colors
heatmap_layer.gradient = ['white','green','blue']

In [63]:
embed_minimal_html('./Visuals/commuteHeatMap.html', views=[fig])

## Regression Analysis

In [68]:
import statsmodels.formula.api as sm
data_df = pd.read_csv(masterDataClean_csv)

result = sm.ols(formula="valuation ~ sqft + commuteTime", data=data_df).fit()
print(result.params)

Intercept      250139.857625
sqft              308.018961
commuteTime    -15954.874641
dtype: float64


## Crime Analysis

In [ ]:
# Kat's section to add 20x20 grid to Austin map

#Cleaning up the crime data
crime_df = crime_df.drop(columns=['Unnamed: 0'])
crime_df = crime_df.dropna()

In [ ]:
#Setting up the bins and labels

xBin = [29.9849664775403, 30.0139206956278, 30.0428749137153, 30.0718291318028, 30.1007833498903, 30.1297375679778, 30.1586917860653, 30.1876460041528, 30.2166002222403, 30.2455544403278, 30.2745086584153, 30.3034628765028, 30.3324170945903, 30.3613713126778, 30.3903255307653, 30.4192797488528, 30.4482339669403, 30.4771881850278, 30.5061424031153, 30.5350966212028, 30.5640508392903]
yBin = [-98.0300375832746, -98.0010750258248, -97.972112468375, -97.9431499109252, -97.9141873534754, -97.8852247960256, -97.8562622385758, -97.827299681126, -97.7983371236762, -97.7693745662264, -97.7404120087766, -97.7114494513268, -97.682486893877, -97.6535243364272, -97.6245617789774, -97.5955992215276, -97.5666366640778, -97.537674106628, -97.5087115491782, -97.4797489917284, -97.4507864342786]
xRegion = ["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T"]
yRegion = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20']

In [ ]:
# Adding Region to masterData dataframe

address_df = pd.read_csv(masterDataCLEAN_csv)

address_df["xRegion"] = pd.cut(address_df["alat"], xBin, labels=xRegion)
address_df["yRegion"] = pd.cut(address_df["alon"], yBin, labels=yRegion)

address_df['Region'] = address_df['xRegion'].astype(str) + address_df['yRegion'].astype(str)

address_df = address_df.drop(columns=['xRegion', 'yRegion'])

In [ ]:
# Adding Region to crime dataframe

crime_df["xRegion"] = pd.cut(crime_df["lat"], xBin, labels=xRegion)
crime_df["yRegion"] = pd.cut(crime_df["lng"], yBin, labels=yRegion)

crime_df['Region'] = crime_df['xRegion'].astype(str) + crime_df['yRegion'].astype(str)

crime_df = crime_df.drop(columns=['xRegion', 'yRegion'])

In [ ]:
# Merging crime dataframe with group severity numbers

crime_comb = pd.merge(crime_df, crimeSev_df[['Offense Desc', 'Severity']], how = 'left', on = 'Offense Desc')

In [ ]:
# Giving each region a 'Severity' rating by summing up weights of different crimes

crimeRegion = pd.DataFrame(crime_comb.groupby(by='Region')['Severity'].sum())
crimeRegion = crimeRegion.reset_index(level=0)

In [ ]:
crime_data = pd.merge(address_df, crimeRegion, how='left', on='Region')